In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [4]:
def giveGeneralInfo(df):
  print("Columns: ", df.columns.tolist(),"\n")
  print("INFORMATION ABOUT DATA")
  print(df.info(),"\n")
  print("NAN COUNT")
  print(df.isna().sum())

In [5]:
test = pd.read_csv("/content/drive/MyDrive/spaceship/kaggle/test.csv")
train = pd.read_csv("/content/drive/MyDrive/spaceship/kaggle/train.csv")

Every passengerId is unique.

In [6]:
giveGeneralInfo(train)

Columns:  ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'] 

INFORMATION ABOUT DATA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtype

In [9]:
train.loc[train["HomePlanet"].isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
59,0064_02,NaN,True,E/3/S,TRAPPIST-1e,33.0,False,0.0,0.0,NaN,0.0,0.0,Colatz Keen,True
113,0119_01,NaN,False,A/0/P,TRAPPIST-1e,39.0,False,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,False
186,0210_01,NaN,True,D/6/P,55 Cancri e,24.0,False,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,True
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False
8613,9194_01,NaN,False,E/603/S,55 Cancri e,53.0,False,0.0,4017.0,0.0,13.0,3147.0,NaN,False
8666,9248_01,NaN,False,F/1792/S,55 Cancri e,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,True
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False


In [ ]:
for column  in train.columns:
  print(column)
  unique = train[column].unique()
  print(unique)
  print("Length Unique",len(unique))
  print()

VIP olanların FoodCourt ortalaması 1811, olmayanların ortalaması 426

VIP olanların FoodCourt maliyeti genelde daha yüksek, bundan dolayı ortalamasını alıp NaN VIP değerleri için bu ortalamadan daha büyük değerleri olan satırlardaki VIP değerini True yaptım. Kalanlar False.

In [ ]:
def splitCabin(df):
  df[["Cabin1","Cabin2","Cabin3"]] = df["Cabin"].str.split("/", expand = True)
  df = df.drop("Cabin",axis = 1)
  return df

def splitPassengerId(df):
  df[["PassengerId1","PassengerId2"]] = df["PassengerId"].str.split("_", expand = True)
  df = df.drop("PassengerId", axis = 1)
  return df

# df.loc[df["VIP"] == True].mean()  -> 1811
# df.loc[df["VIP"] == False].mean() -> 426
def fillVIPValues(df):
  temp = df.loc[df["VIP"] == True]
  vip_true_mean = temp["FoodCourt"].mean() # 1811.3937823834196
  # print(df["VIP"].isna().sum())
  df["VIP"].loc[ (df["VIP"].isna()) & (df["FoodCourt"] > vip_true_mean) ] = True # 9 değer True oldu.
  # print(df["VIP"].isna().sum())
  df["VIP"] = df["VIP"].fillna(False)
  return df

def fillDestinationValues(df): # Destination'daki NaN verileri oransal olarak en çok olan değer ile doldurdum.
  destination_list = df["Destination"].unique()
  value = "TRAPPIST-1e"
  if value in destination_list:
    df["Destination"] = df["Destination"].fillna(value)
  return df

# True False oranına göre NaN'ları dolduruyorum.
def fillCryoSleep(df):
  temp = df["CryoSleep"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 217
  temp2 = df["CryoSleep"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  true_number = round(nan_count * true_ratio)
  false_number = round(nan_count * false_ratio)
  total = true_number + false_number
  nan_indices = df['CryoSleep'].index[df['CryoSleep'].isna()] # NaN değerlerin indexleri
  if total == len(nan_indices):
    nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
    df.loc[nan_indices_shuffled[:true_number], 'CryoSleep'] = True
    df.loc[nan_indices_shuffled[true_number:], 'CryoSleep'] = False
  return df

# NaN veriler oransal olarak dolduruldu.
def fillHomePlanet(df):
  temp = df["HomePlanet"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 201
  ratios = df["HomePlanet"].value_counts(normalize = True, dropna = True)
  earth_ratio = ratios["Earth"]
  europa_ratio = ratios["Europa"]
  mars_ratio = ratios["Mars"]
  earth_number = round(nan_count * earth_ratio)
  europa_number = round(nan_count * europa_ratio)
  mars_number = round(nan_count * mars_ratio)

  nan_indices = df["HomePlanet"].index[df["HomePlanet"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  df.loc[nan_indices_shuffled[:earth_number], "HomePlanet"] = "Earth"
  df.loc[nan_indices_shuffled[earth_number:earth_number + europa_number], "HomePlanet"] = "Europa"
  df.loc[nan_indices_shuffled[earth_number + europa_number : earth_number + europa_number + mars_number], "HomePlanet"] = "Mars"
  return df

def fillRoomService(df): # 181 nan values    0, x > 0, nan
  column = df.loc[df["RoomService"] > 0]["RoomService"]
  mean = column.mean() # 651.6323679727427
  df["RoomServiceBool"] = np.nan
  df.loc[df["RoomService"] > 0, "RoomServiceBool"] = True
  df.loc[df["RoomService"] == 0, "RoomServiceBool"] = False
  temp = df["RoomServiceBool"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 181
  temp2 = df["RoomServiceBool"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  nan_indices = df["RoomServiceBool"].index[df["RoomServiceBool"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  true_number = round(true_ratio * nan_count)
  false_number = round(false_ratio * nan_count)
  total = true_number + false_number
  if total == nan_count:
    df.loc[nan_indices_shuffled[:true_number], "RoomService"] = mean
    df.loc[nan_indices_shuffled[true_number:], "RoomService"] = 0
  df = df.drop("RoomServiceBool", axis = 1)
  return df

def fillFoodCourt(df): # 181 nan values    0, x > 0, nan
  column = df.loc[df["FoodCourt"] > 0]["FoodCourt"]
  mean = column.mean() # 651.6323679727427
  df["FoodCourtBool"] = np.nan
  df.loc[df["FoodCourt"] > 0, "FoodCourtBool"] = True
  df.loc[df["FoodCourt"] == 0, "FoodCourtBool"] = False
  temp = df["FoodCourtBool"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 183
  # print(nan_count)
  temp2 = df["FoodCourtBool"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  nan_indices = df["FoodCourtBool"].index[df["FoodCourtBool"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  true_number = round(true_ratio * nan_count)
  false_number = round(false_ratio * nan_count)
  total = true_number + false_number
  if total == nan_count:
    df.loc[nan_indices_shuffled[:true_number], "FoodCourt"] = mean
    df.loc[nan_indices_shuffled[true_number:], "FoodCourt"] = 0
  df = df.drop("FoodCourtBool", axis = 1)
  return df

def fillShoppingMall(df): # 181 nan values
  column = df.loc[df["ShoppingMall"] > 0]["ShoppingMall"]
  mean = column.mean() # 651.6323679727427
  df["ShoppingMallBool"] = np.nan
  df.loc[df["ShoppingMall"] > 0, "ShoppingMallBool"] = True
  df.loc[df["ShoppingMall"] == 0, "ShoppingMallBool"] = False
  temp = df["ShoppingMallBool"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 183
  # print(nan_count)
  temp2 = df["ShoppingMallBool"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  nan_indices = df["ShoppingMallBool"].index[df["ShoppingMallBool"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  true_number = round(true_ratio * nan_count)
  false_number = round(false_ratio * nan_count)
  total = true_number + false_number
  if total == nan_count:
    df.loc[nan_indices_shuffled[:true_number], "ShoppingMall"] = mean
    df.loc[nan_indices_shuffled[true_number:], "ShoppingMall"] = 0
  df = df.drop("ShoppingMallBool", axis = 1)
  return df

def fillSpa(df): # 181 nan values
  column = df.loc[df["Spa"] > 0]["Spa"]
  mean = column.mean() # 651.6323679727427
  df["SpaBool"] = np.nan
  df.loc[df["Spa"] > 0, "SpaBool"] = True
  df.loc[df["Spa"] == 0, "SpaBool"] = False
  temp = df["SpaBool"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 183
  # print(nan_count)
  temp2 = df["SpaBool"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  nan_indices = df["SpaBool"].index[df["SpaBool"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  true_number = round(true_ratio * nan_count)
  false_number = round(false_ratio * nan_count)
  total = true_number + false_number
  if total == nan_count:
    df.loc[nan_indices_shuffled[:true_number], "Spa"] = mean
    df.loc[nan_indices_shuffled[true_number:], "Spa"] = 0
  df = df.drop("SpaBool", axis = 1)
  return df

def fillVRDeck(df): # 181 nan values
  column = df.loc[df["VRDeck"] > 0]["VRDeck"]
  mean = column.mean() # 651.6323679727427
  df["VRDeckBool"] = np.nan
  df.loc[df["VRDeck"] > 0, "VRDeckBool"] = True
  df.loc[df["VRDeck"] == 0, "VRDeckBool"] = False
  temp = df["VRDeckBool"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 183
  # print(nan_count)
  temp2 = df["VRDeckBool"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  nan_indices = df["VRDeckBool"].index[df["VRDeckBool"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  true_number = round(true_ratio * nan_count)
  false_number = round(false_ratio * nan_count)
  total = true_number + false_number
  if total == nan_count:
    df.loc[nan_indices_shuffled[:true_number], "VRDeck"] = mean
    df.loc[nan_indices_shuffled[true_number:], "VRDeck"] = 0
  df = df.drop("VRDeckBool", axis = 1)
  return df

def fillAge(df): # 181 nan values
  column = df.loc[df["Age"] > 0]["Age"]
  mean = column.mean() # 651.6323679727427
  df["AgeBool"] = np.nan
  df.loc[df["Age"] > 0, "AgeBool"] = True
  df.loc[df["Age"] == 0, "AgeBool"] = False
  temp = df["AgeBool"].value_counts(normalize = False, dropna = False)
  nan_count = temp[np.nan] # 183
  # print(nan_count)
  temp2 = df["AgeBool"].value_counts(normalize = True, dropna = True)
  true_ratio = temp2[True]
  false_ratio = temp2[False]
  nan_indices = df["AgeBool"].index[df["AgeBool"].isna()]
  nan_indices_shuffled = np.random.choice(nan_indices, size=len(nan_indices), replace=False)
  true_number = round(true_ratio * nan_count)
  false_number = round(false_ratio * nan_count)
  total = true_number + false_number
  if total == nan_count:
    df.loc[nan_indices_shuffled[:true_number], "Age"] = mean
    df.loc[nan_indices_shuffled[true_number:], "Age"] = 0
  df = df.drop("AgeBool", axis = 1)
  return df

In [ ]:
# label_encoder = LabelEncoder()
def labelEncoder(df):
  for column in df.columns:
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])
    # if column == "PassengerId1":
    #   passengerid1 = label_encoder.inverse_transform(df[column])
    # elif column == "PassengerId2":
    #   passengerid2 = label_encoder.inverse_transform(df[column])
  return df

def createSubmissionCsv(y_pred, test_data):
  submission_df = pd.DataFrame()
  submission_df["PassengerId"] = test_data["PassengerId"]
  submission_df["Transported"] = y_pred
  submission_df = submission_df.set_index("PassengerId")
  return submission_df

In [ ]:
def preprocessing(df):
  df = splitCabin(df)
  df = splitPassengerId(df)
  df = fillVIPValues(df)
  df = fillDestinationValues(df)
  df = fillCryoSleep(df)
  df = fillHomePlanet(df)
  df = fillRoomService(df)
  df = fillFoodCourt(df)
  df = fillShoppingMall(df)
  df = fillSpa(df)
  df = fillVRDeck(df)
  df = fillAge(df)
  return df

Sayısal değerler için hizmet var mı yok mu kontrol et. Bunu oranla. Oranlama bittikten sonra alışveriş olan değerleri ortalama ile doldur.

Bazı isimler 2 kere kullanılmış

In [ ]:
def encodeTrain(df):
  encode_list = ["HomePlanet", "CryoSleep", "Destination", "VIP", "Name", "Transported", "Cabin1", "Cabin2", "Cabin3", "PassengerId1", "PassengerId2"]
  others = df.drop(encode_list, axis = 1)
  df = labelEncoder(df[encode_list])
  final_df = pd.concat([df,others], axis = 1)
  return final_df

def encodeTest(df):
  encode_list = ["HomePlanet", "CryoSleep", "Destination", "VIP", "Name", "Cabin1", "Cabin2", "Cabin3", "PassengerId1", "PassengerId2"]
  others = df.drop(encode_list, axis = 1)
  df = labelEncoder(df[encode_list])
  final_df = pd.concat([df,others], axis = 1)
  return final_df

In [ ]:
train_data = preprocessing(train)
train_data = encodeTrain(train_data)

In [ ]:
test_data = preprocessing(test)
test_data["PassengerId"] = test_data["PassengerId1"].astype(str) + "_" +test_data["PassengerId2"].astype(str)
# passengerId = test_data[""]
test_data = encodeTest(test_data)
# test_data["PassengerId"] = test_data["PassengerId1"].astype(str) + "_" +test_data["PassengerId2"].astype(str)

In [ ]:
X = train_data.drop("Transported", axis = 1)
y = train_data["Transported"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def hyperparameterOptimizationForXGBoost(X_train, y_train):
  model = XGBClassifier()

  max_depth_list = [3,5,7] # Karar ağacıların maksimum derinliği. 3-10 arası bir değer genellikle önerilir.
  learning_rate_list = [0.1, 0.01, 0.05] # Her ağaç oluşturma işleminde kullanılan öğrenme oranı. 0.01-0.1 arası bir değer genellikle önerilir.
  n_estimators_list = [100,250,500,1000] # Oluşturulacak ağaç sayısı. 50-500 arası bir değer genellikle önerilir. 50,100,250,500,

  param_grid = {
      "learning_rate" : [0.1, 0.01, 0.001],
      "max_depth" : [3,5,7],
      "n_estimators" : [100,250,500,1000]
      }


  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
  grid_search.fit(X_train,y_train)

  print("XGBoost")
  print("En iyi hiperparametreler:", grid_search.best_params_)
  print("En iyi doğruluk oranı:", grid_search.best_score_)
  return grid_search.best_estimator_ #best model

In [ ]:
def hyperparameterOptimizationForRandomForestClassifier(X_train, y_train):
  param_grid = {
      'n_estimators': [100, 250, 500],
      'max_depth': [None, 5, 10],
      'min_samples_split': [2, 3, 5],
      'min_samples_leaf': [2, 3, 4]
  }

  model = RandomForestClassifier()
  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
  grid_search.fit(X, y)

  print("RandomForestClassifier")
  print("En iyi hiperparametreler:", grid_search.best_params_)
  print("En iyi doğruluk oranı:", grid_search.best_score_)
  return grid_search.best_estimator_ #best model

In [ ]:
def hyperparameterOptimizationForSVC(X_train, y_train):
  model = SVC()

  param_grid = {
      'C': [0.1, 1, 10, 100],
      'gamma': [0.1, 0.01, 0.001, 0.0001],
      'kernel': ['linear', 'rbf', 'sigmoid']
  }

  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
  grid_search.fit(X_train, y_train)

  print("SVC")
  print("En iyi hiperparametreler:", grid_search.best_params_)
  print("En iyi doğruluk oranı:", grid_search.best_score_)
  return grid_search.best_estimator_#best model

In [ ]:
def hyperparametreOptimizationForCatBoost(X_train, y_train):
  model = CatBoostClassifier()
  param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'depth': [4, 6, 8],
    'iterations': [100, 200, 300],
    "verbose" : [False]
  }
  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
  grid_search.fit(X_train, y_train)
  print("En iyi hiperparametreler:", grid_search.best_params_)
  print("En iyi doğruluk oranı:", grid_search.best_score_)
  return grid_search.best_estimator_

In [ ]:
def hyperparametreOptimizationForLGBMClassifier(X_train, y_train):
  model = LGBMClassifier()
  param_grid = {
      'learning_rate': [0.1, 0.01, 0.001],
      'max_depth': [3, 5, 7],
      'n_estimators': [100, 200, 300]
  }

  # Grid SearchCV ile hiperparametre optimizasyonu
  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
  grid_search.fit(X_train, y_train)

  # En iyi hiperparametre değerlerini ve doğruluk oranını yazdırma
  print("En iyi hiperparametreler:", grid_search.best_params_)
  print("En iyi doğruluk oranı:", grid_search.best_score_)
  return grid_search.best_estimator_

In [ ]:
def getBestModel(X_train, X_test, y_train, y_test):
  accuracy_list = []
  model_accuracy_dict = {}

  # XGBCLASSIFIER
  xgbClassifier = XGBClassifier()
  xgbClassifier.fit(X_train,y_train)
  pred_xgb = xgbClassifier.predict(X_test)
  acc_xgb = accuracy_score(y_test, pred_xgb)
  # accuracy_list.append(acc_xgb)
  model_accuracy_dict.update({acc_xgb : "XGBClassifier"})
  print("XGBClassifier accuracy:", acc_xgb)

  # RANDOMFOREST
  randomForest = RandomForestClassifier()
  randomForest.fit(X_train, y_train)
  pred_randomForest = randomForest.predict(X_test)
  acc_randomForest = accuracy_score(y_test, pred_randomForest)
  # accuracy_list.append(acc_randomForest)
  model_accuracy_dict.update({acc_randomForest : "RandomForest"})
  print("RandomForest accuracy:", acc_randomForest)

  # SVC
  svc = SVC()
  svc.fit(X_train, y_train)
  pred_svc = svc.predict(X_test)
  acc_svc = accuracy_score(y_test, pred_svc)
  # accuracy_list.append(acc_svc)
  model_accuracy_dict.update({acc_svc : "SVC"})
  print("SVC accuracy:", acc_svc)

  # CATBOOST
  catboost = CatBoostClassifier()
  catboost.fit(X_train, y_train, verbose=False)
  pred_catboost = catboost.predict(X_test)
  acc_catboost = accuracy_score(y_test, pred_catboost)
  model_accuracy_dict.update({acc_catboost : "Catboost"})
  print("Catboost Accuracy:", acc_catboost)

  # KNeighborsClassifier
  kneighbors = KNeighborsClassifier(n_neighbors=3)
  kneighbors.fit(X_train, y_train)
  pred_kneighbors = kneighbors.predict(X_test)
  acc_kneighbors = accuracy_score(y_test, pred_kneighbors)
  model_accuracy_dict.update({acc_kneighbors : "KNeighbors"})
  print("KNeighbors Accuracy:", acc_kneighbors)

  # Gaussian Naive Bayes
  nb = GaussianNB()
  nb.fit(X_train, y_train)
  pred_nb = nb.predict(X_test)
  acc_nb = accuracy_score(y_test, pred_nb)
  model_accuracy_dict.update({acc_nb : "GaussianNB"})
  print("GauusianNB accuracy:", acc_nb)

  # LGBMClassifier
  lgbm = LGBMClassifier()
  lgbm.fit(X_train, y_train)
  pred_lgbm = lgbm.predict(X_test)
  acc_lgbm = accuracy_score(y_test, pred_lgbm)
  model_accuracy_dict.update({acc_lgbm : "LGBMClassifier"})
  print("LGBMClassifier accuracy:", acc_lgbm)

  # GradientBoostClassifier
  gradient = GradientBoostingClassifier()
  gradient.fit(X_train, y_train)
  pred_gradient = gradient.predict(X_test)
  acc_gradient = accuracy_score(y_test, pred_gradient)
  model_accuracy_dict.update({acc_gradient : "GradientBoostClassifier"})
  print("GradientBoostClassifier accuracy:", acc_gradient)

  # AdaBoostClassifier
  ada = AdaBoostClassifier()
  ada.fit(X_train, y_train)
  pred_ada = ada.predict(X_test)
  acc_ada = accuracy_score(y_test, pred_ada)
  model_accuracy_dict.update({acc_ada : "AdaBoostClassifier"})
  print("AdaBoostClassifier accuracy:", acc_ada)

  accuracy_list = model_accuracy_dict.keys()
  best_result = max(accuracy_list)

  best_model = model_accuracy_dict.get(best_result)
  print(best_model)

  return best_model

In [ ]:
best_model_name = getBestModel(X_train, X_test, y_train, y_test)
best_model_name

XGBClassifier accuracy: 0.7860839562967222
RandomForest accuracy: 0.7837837837837838
SVC accuracy: 0.7602070155261644
Catboost Accuracy: 0.7941345600920069
KNeighbors Accuracy: 0.721679125934445
GauusianNB accuracy: 0.7084531339850488
LGBMClassifier accuracy: 0.7947096032202415
GradientBoostClassifier accuracy: 0.78953421506613
AdaBoostClassifier accuracy: 0.7722829212190915
LGBMClassifier


'LGBMClassifier'

In [ ]:
best_model = hyperparametreOptimizationForLGBMClassifier(X_train, y_train)
best_model

En iyi hiperparametreler: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 300}
En iyi doğruluk oranı: 0.8029910842680472


LGBMClassifier(learning_rate=0.01, max_depth=7, n_estimators=300)

In [ ]:
y_pred = best_model.predict(test_data.drop("PassengerId", axis = 1))

In [ ]:
y_pred = list(map(bool, y_pred))
y_pred

In [ ]:
# if best_model_name == "XgbClassifier":
#   hyperparameterOptimizationForXGBoost()

In [ ]:
submission_df = createSubmissionCsv(y_pred,test_data)

In [ ]:
submission_df

In [ ]:
submission_df.to_csv(f"/content/drive/MyDrive/spaceship/{best_model_name}_results.csv")